In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
from datetime import datetime, timedelta
np.random.seed(1)

In [2]:
parse_dates=['date_created','date_deadline','date_launched','date_ended']
df_2019 = pd.read_csv('./kickstarter_2019.csv', parse_dates=parse_dates)

In [3]:
df_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44158 entries, 0 to 44157
Data columns (total 52 columns):
backers_count               44158 non-null int64
blurb                       44158 non-null object
category                    44158 non-null object
category_name               44158 non-null object
category_specific           44158 non-null object
converted_pledged_amount    44158 non-null int64
country                     44158 non-null object
country_displayable_name    44158 non-null object
created_at                  44158 non-null int64
date_created                44158 non-null datetime64[ns]
creator                     44158 non-null object
currency                    44158 non-null object
currency_symbol             44158 non-null object
currency_trailing_code      44158 non-null bool
current_currency            44158 non-null object
deadline                    44158 non-null int64
date_deadline               44158 non-null datetime64[ns]
disable_communication       441

In [70]:
print(df_2019['category'][25])

{'id': 34, 'name': 'Tabletop Games', 'slug': 'games/tabletop games', 'position': 6, 'parent_id': 12, 'color': 51627, 'urls': {'web': {'discover': 'http://www.kickstarter.com/discover/categories/games/tabletop%20games'}}}


In [4]:
df_2019.groupby('category_name')['category_specific'].value_counts().head()

category_name  category_specific     
3D Printing    technology/3d printing     108
Academic       publishing/academic        152
Accessories    fashion/accessories       1584
Action         film & video/action        101
Animals        photography/animals         25
Name: category_specific, dtype: int64

In [21]:
df_2019['usd_pledged'].head(15)

0       719.756813
1        82.000000
2      1147.105655
3      7415.000000
4         1.000000
5      1159.000000
6         1.121949
7     11554.250000
8       840.036183
9      1681.000000
10        1.122209
11      846.000000
12     2234.861841
13     1373.789594
14     5032.604125
Name: usd_pledged, dtype: float64

In [23]:
df_2019['converted_pledged_amount'].head(15)

0       709
1        82
2      1153
3      7415
4         1
5      1159
6         1
7     11554
8       842
9      1681
10        1
11      846
12     2253
13     1351
14     4990
Name: converted_pledged_amount, dtype: int64

In [6]:
df_2019['category_name'].nunique()

159

In [7]:
df_2019['category_specific'].nunique()

169

In [115]:
category_df = df_2019['category_specific'].groupby(df_2019['category_name'])
c_df1 = category_df.describe()
c_df1

count unique                     top  freq
category_name                                           
3D Printing     108      1  technology/3d printing   108
Academic        152      1     publishing/academic   152
Accessories    1584      1     fashion/accessories  1584
Action          101      1     film & video/action   101
Animals          25      1     photography/animals    25
...             ...    ...                     ...   ...
Woodworking     157      1      crafts/woodworking   157
Workshops        14      1         dance/workshops    14
World Music     232      1       music/world music   232
Young Adult     148      1  publishing/young adult   148
Zines           152      1        publishing/zines   152

[159 rows x 4 columns]

In [128]:
c_df1[c_df1['unique'] > 1]

# 조회 결과 아래의 Anthologies, Comdedy, Events 등의 category는 더 큰 분류에서 보면 1개 이상이다
# Comedy를 예로 들면, music/comedy, publishing/comedy, theater/comedy, film & video/comedy 이렇게 4개의 큰 카테고리들이 다 comedy라는 subcategory가 있음
# Spaces도 예로 들면, dance/spaces, food/spaces, theater/spaces에 각 spaces라는 subcategory가 있다

count unique                        top freq
category_name                                             
Anthologies     416      2         comics/anthologies  243
Comedy          581      4        film & video/comedy  362
Events           93      2                food/events   52
Experimental    148      2  film & video/experimental   85
Festivals       160      2          theater/festivals  115
Spaces          157      3                food/spaces   91
Web             480      2             technology/web  279

In [121]:
test2 = df_2019['category_name'].groupby(df_2019['category_specific'])

In [124]:
c_df2 = test2.describe()
c_df2

count unique             top  freq
category_specific                                    
art                 1001      1             Art  1001
art/ceramics          83      1        Ceramics    83
art/conceptual art   131      1  Conceptual Art   131
art/digital art      480      1     Digital Art   480
art/illustration    1503      1    Illustration  1503
...                  ...    ...             ...   ...
theater/festivals    115      1       Festivals   115
theater/immersive     64      1       Immersive    64
theater/musical      160      1         Musical   160
theater/plays        252      1           Plays   252
theater/spaces        36      1          Spaces    36

[169 rows x 4 columns]

In [129]:
#c_df2[c_df2['count'] != c_df2['freq']]
c_df2[c_df2['unique'] > 1] #이건 category_specific이랑 category_name 개수가 맞다

Empty DataFrame
Columns: [count, unique, top, freq]
Index: []

In [96]:
df_2019[df_2019['category_specific'] == 'art'].shape

(1001, 52)

In [72]:
np.unique(df_2019['category_specific'].values)

#어떤 프로젝트들은 category_name, category_specific 둘 다 같은 것이 있다; 예)art, comics, crafts 등
# 이런 프로젝트들은 main category는 설정 했지만 subcategory는 설정 안 했기 때문

array(['art', 'art/ceramics', 'art/conceptual art', 'art/digital art',
       'art/illustration', 'art/installations', 'art/mixed media',
       'art/painting', 'art/performance art', 'art/public art',
       'art/sculpture', 'art/social practice', 'art/textiles',
       'art/video art', 'comics', 'comics/anthologies',
       'comics/comic books', 'comics/events', 'comics/graphic novels',
       'comics/webcomics', 'crafts', 'crafts/candles', 'crafts/crochet',
       'crafts/diy', 'crafts/embroidery', 'crafts/glass',
       'crafts/knitting', 'crafts/pottery', 'crafts/printing',
       'crafts/quilts', 'crafts/stationery', 'crafts/weaving',
       'crafts/woodworking', 'dance', 'dance/performances',
       'dance/residencies', 'dance/spaces', 'dance/workshops', 'design',
       'design/architecture', 'design/civic design',
       'design/graphic design', 'design/interactive design',
       'design/product design', 'design/typography', 'fashion',
       'fashion/accessories', 'fashion/a

In [79]:
test = df_2019.groupby('category_name')
test.groups.keys()

dict_keys(['3D Printing', 'Academic', 'Accessories', 'Action', 'Animals', 'Animation', 'Anthologies', 'Apparel', 'Apps', 'Architecture', 'Art', 'Art Books', 'Audio', 'Bacon', 'Blues', 'Calendars', 'Camera Equipment', 'Candles', 'Ceramics', "Children's Books", 'Childrenswear', 'Chiptune', 'Civic Design', 'Classical Music', 'Comedy', 'Comic Books', 'Comics', 'Community Gardens', 'Conceptual Art', 'Cookbooks', 'Country & Folk', 'Couture', 'Crafts', 'Crochet', 'DIY', 'DIY Electronics', 'Dance', 'Design', 'Digital Art', 'Documentary', 'Drama', 'Drinks', 'Electronic Music', 'Embroidery', 'Events', 'Experimental', 'Fabrication Tools', 'Faith', 'Family', 'Fantasy', "Farmer's Markets", 'Farms', 'Fashion', 'Festivals', 'Fiction', 'Film & Video', 'Fine Art', 'Flight', 'Food', 'Food Trucks', 'Footwear', 'Gadgets', 'Games', 'Gaming Hardware', 'Glass', 'Graphic Design', 'Graphic Novels', 'Hardware', 'Hip-Hop', 'Horror', 'Illustration', 'Immersive', 'Indie Rock', 'Installations', 'Interactive Design'

In [35]:
df_2019.loc[df_2019['category_specific'].apply(lambda x: x.count(x)) > 1]

backers_count                                              blurb  \
0                 31  Un livre enfant pour l’apprentissage des émoti...   
1                  5  Rewarding you for all of the things you do and...   
2                 31  Follow unwitting heroes Bluebird and Kit as th...   
3                 72  Genesis follows a mother and son that live tog...   
4                  1  A monthly newsletter that contains local testi...   
...              ...                                                ...   
44153             89     Adorable cat enamel pins for crazy cat ladies.   
44154            129                          LGBTQ Rainbow Enamel Pins   
44155              2  Do you want to create social media content abo...   
44156             10  God has blessed me with more than 60,000 downl...   
44157              3  Artist Designed Urban Tote\nPunKin Apparel by ...   

                                                category     category_name  \
0      {'id': 46, 'name': "Children's Books", 'slug':...  Children's Books   
1      {'id': 332, 'name': 'Apps', 'slug': 'technolog...              Apps   
2      {'id': 252, 'name': 'Graphic Novels', 'slug': ...    Graphic Novels   
3      {'id': 301, 'name': 'Science Fiction', 'slug':...   Science Fiction   
4      {'id': 359, 'name': 'Print', 'slug': 'journali...             Print   
...                                                  ...               ...   
44153  {'id': 262, 'name': 'Accessories', 'slug': 'fa...       Accessories   
44154  {'id': 262, 'name': 'Accessories', 'slug': 'fa...       Accessories   
44155  {'id': 361, 'name': 'Web', 'slug': 'journalism...               Web   
44156  {'id': 318, 'name': 'Faith', 'slug': 'music/fa...             Faith   
44157  {'id': 289, 'name': 'Textiles', 'slug': 'art/t...          Textiles   

                  category_specific  converted_pledged_amount country  \
0       publishing/children's books                       709      FR   
1                   technology/apps                        82      US   
2             comics/graphic novels                      1153      GB   
3      film & video/science fiction                      7415      US   
4                  journalism/print                         1      US   
...                             ...                       ...     ...   
44153           fashion/accessories                      2727      AU   
44154           fashion/accessories                      2799      GB   
44155                journalism/web                         2      DE   
44156                   music/faith                      1045      US   
44157                  art/textiles                        93      US   

      country_displayable_name  created_at        date_created  ...  \
0                       France  1561554849 2019-06-26 13:14:09  ...   
1            the United States  1553176181 2019-03-21 13:49:41  ...   
2           the United Kingdom  1546607652 2019-01-04 13:14:12  ...   
3            the United States  1571610662 2019-10-20 22:31:02  ...   
4            the United States  1566598445 2019-08-23 22:14:05  ...   
...                        ...         ...                 ...  ...   
44153                Australia  1552650822 2019-03-15 11:53:42  ...   
44154       the United Kingdom  1554826932 2019-04-09 16:22:12  ...   
44155                  Germany  1554806893 2019-04-09 10:48:13  ...   
44156        the United States  1553434574 2019-03-24 13:36:14  ...   
44157        the United States  1558034415 2019-05-16 19:20:15  ...   

               date_ended static_usd_rate  \
0     2019-08-25 14:46:37        1.140661   
1     2019-09-10 21:56:21        1.000000   
2     2019-03-11 18:27:42        1.290333   
3     2019-11-21 18:14:53        1.000000   
4     2019-08-29 14:59:49        1.000000   
...                   ...             ...   
44153 2019-04-22 02:00:20        0.714847   
44154 2019-05-11 16:04:18        1.305500   
44155 2019-05-02 06:53:15        1

In [98]:
df_2019['spotlight'].value_counts()

True     30002
False    14156
Name: spotlight, dtype: int64

In [97]:
len(df_2019[df_2019['spotlight'] == True])

30002

In [99]:
df_2019['staff_pick'].value_counts()

False    38448
True      5710
Name: staff_pick, dtype: int64

In [4]:
df_2019.columns

Index(['backers_count', 'blurb', 'category', 'category_name',
       'category_specific', 'converted_pledged_amount', 'country',
       'country_displayable_name', 'created_at', 'date_created', 'creator',
       'currency', 'currency_symbol', 'currency_trailing_code',
       'current_currency', 'deadline', 'date_deadline',
       'disable_communication', 'friends', 'fx_rate', 'goal', 'id',
       'is_backing', 'is_starrable', 'is_starred', 'launched_at',
       'date_launched', 'date_launched_year', 'location', 'location_name',
       'name', 'permissions', 'photo', 'photo_link', 'pledged', 'profile',
       'slug', 'source_url', 'spotlight', 'staff_pick', 'state',
       'state_changed_at', 'date_ended', 'static_usd_rate', 'urls', 'urls_web',
       'url_project', 'url_reward', 'usd_pledged', 'usd_type',
       'fund_recruitment_period', 'actual_time_taken'],
      dtype='object')

In [ ]:
category_name
category_specific
location_name
country_displayable_name
goal
created_at (or date_created)
launched_at (or date_launched)
deadline (or date_deadline)
date_launched_year
disable_communication
is_starrable
fund_recruitment_period : date_deadline-date_launched
or set_fundraising_period -- in seconds : deadline-launched_at

+
date_launched_month
date_launched_day (?)
launching_delay_time : launched_at-created_at or date_launched-date_created
goal/set_fundraising_period : 지정한 펀딩기간 대비 목표 모금액
or set_fundraising_period/goal : 목표 모금액 대비 지정한 펀딩기간 -- 이건 비상식적인 것 같음

In [16]:
#set_fundraising_period -- in seconds : deadline-launched_at
set_fundraising_period = df_2019['deadline']-df_2019['launched_at']
set_fundraising_period

0        5184000
1        2592000
2        2156400
3        2595600
4        5184000
          ...   
44153    2592000
44154    2592000
44155    1728000
44156    5011887
44157    2657079
Length: 44158, dtype: int64

In [17]:
#date_launched_month
date_launched_month = df_2019['date_launched'].dt.month
date_launched_month

0         6
1         8
2         2
3        10
4         8
         ..
44153     3
44154     4
44155     4
44156     4
44157     6
Name: date_launched, Length: 44158, dtype: int64

In [18]:
#date_launched_day
date_launched_day = df_2019['date_launched'].dt.day
date_launched_day

0        26
1        11
2        14
3        22
4        27
         ..
44153    23
44154    11
44155    12
44156    23
44157    15
Name: date_launched, Length: 44158, dtype: int64

In [19]:
#launching_delay_time : launched_at-created_at (or date_launched-date_created)
launching_delay_time = df_2019['launched_at']-df_2019['created_at'] # in seconds
launching_delay_time

0            5548
1        12384398
2         3564810
3          153831
4          341123
           ...   
44153      655598
44154      171726
44155      245102
44156     2599879
44157     2568846
Length: 44158, dtype: int64

In [20]:
launching_delay_days = df_2019['date_launched']-df_2019['date_created'] # in days and hours
launching_delay_days

0         0 days 01:32:28
1       143 days 08:06:38
2        41 days 06:13:30
3         1 days 18:43:51
4         3 days 22:45:23
               ...       
44153     7 days 14:06:38
44154     1 days 23:42:06
44155     2 days 20:05:02
44156    30 days 02:11:19
44157    29 days 17:34:06
Length: 44158, dtype: timedelta64[ns]

In [22]:
#goal/set_fundraising_period : 지정한 펀딩기간 대비 목표 모금액
target_goal_period_rate = df_2019['goal']/set_fundraising_period
target_goal_period_rate

0        0.000116
1        0.011574
2        0.000363
3        0.002697
4        0.001929
           ...   
44153    0.000212
44154    0.000386
44155    0.004051
44156    0.003991
44157    0.000263
Length: 44158, dtype: float64

In [25]:
df_2019.columns.get_loc('fund_recruitment_period')+1

51

In [27]:
df_2019.insert(51, 'set_fundraising_period', set_fundraising_period)

In [29]:
df_2019['date_launched_month'] = date_launched_month

In [30]:
df_2019['date_launched_day'] = date_launched_day

In [31]:
df_2019['launching_delay_time'] = launching_delay_time

In [32]:
df_2019['target_goal_period_rate'] = target_goal_period_rate

In [34]:
df_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44158 entries, 0 to 44157
Data columns (total 57 columns):
backers_count               44158 non-null int64
blurb                       44158 non-null object
category                    44158 non-null object
category_name               44158 non-null object
category_specific           44158 non-null object
converted_pledged_amount    44158 non-null int64
country                     44158 non-null object
country_displayable_name    44158 non-null object
created_at                  44158 non-null int64
date_created                44158 non-null datetime64[ns]
creator                     44158 non-null object
currency                    44158 non-null object
currency_symbol             44158 non-null object
currency_trailing_code      44158 non-null bool
current_currency            44158 non-null object
deadline                    44158 non-null int64
date_deadline               44158 non-null datetime64[ns]
disable_communication       441

In [35]:
df_2019.to_csv('ks_2019_columns_added.csv', index=False, encoding='utf-8')

In [ ]:
# title, subtitle(blurb) 길이

In [ ]:
#ks_2019_crawling.ipynb에서 컬럼 하나 더 ('usd_goal') 추가했다

In [37]:
parse_dates=['date_created','date_deadline','date_launched','date_ended']
df_2019 = pd.read_csv('ks_2019_include_all_state.csv', parse_dates=parse_dates)

In [38]:
df_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44158 entries, 0 to 44157
Data columns (total 58 columns):
backers_count               44158 non-null int64
blurb                       44158 non-null object
category                    44158 non-null object
category_name               44158 non-null object
category_specific           44158 non-null object
converted_pledged_amount    44158 non-null int64
country                     44158 non-null object
country_displayable_name    44158 non-null object
created_at                  44158 non-null int64
date_created                44158 non-null datetime64[ns]
creator                     44158 non-null object
currency                    44158 non-null object
currency_symbol             44158 non-null object
currency_trailing_code      44158 non-null bool
current_currency            44158 non-null object
deadline                    44158 non-null int64
date_deadline               44158 non-null datetime64[ns]
disable_communication       441